# Model Selection

[Resource](https://harvard-iacs.github.io/2018-CS109A/sections/section-4/demo/)

In [3]:
import numpy as np
import pandas as pd
from sklearn import metrics, datasets
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['figure.figsize'] = (13.0, 6.0)

import itertools

from IPython.display import display
pd.set_option("display.max_columns", 999)
pd.set_option("display.width", 500)
sns.set_style("whitegrid")
sns.set_context("talk")

# NYC Car Hire Dataset

Welp... here we are again. This data requires a request for access to the professor's google drive, SO it looks like our only current option is to read through the example for now. Who knows! Maybe they'll accept my Google Drive access request!